In this notebook, we will go over Word2Vec as an application of deep learning. We will then look at using the output of Word2Vec in two ways  
1. Expanding our genre wordlists
2. As features for predicting the story scores

In [1]:
import scala.collection.immutable.TreeSet
import java.util

In [2]:
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/annotate-1.0.jar
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/opennlp-tools-1.6.0.jar
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/models.jar

Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/annotate-1.0.jar
Finished download of annotate-1.0.jar
Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/opennlp-tools-1.6.0.jar
Finished download of opennlp-tools-1.6.0.jar
Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/models.jar
Finished download of models.jar


In [3]:
import nlp.spark.annotate.{ Annotation, Annotator, Document }

In [4]:
import org.apache.spark.ml.classification._
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.{ Pipeline, PipelineModel, Transformer }
import org.apache.spark.ml.regression._
import org.apache.spark.mllib.linalg._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [5]:
val dataframe = sqlContext.read.parquet("stories.parquet").cache()

In [6]:
dataframe.count()

1289

In [7]:
dataframe

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, isTopLevel: double, textLength: int, avgSentenceLengths: double, words: array<string>, numEntities: double, score: double, isStoryPred: string, popularity: string]

Word2Vec in general uses the words that co-occur with a given word ($w$), to represent $w$ as a vector. The problem that occurs with this is the high dimensionality of this vector space. One way to is to create a model to predict the context given a word. If two words have the same predicted context then they are used in similar situations, and we expect them to be semantically similar.

Apache Spark's implementation of Word2Vec uses skip-grams. Skip-grams are a generalization of n-grams.

2-grams of "the quick fox jumped over the lazy dog" are  
"the quick", "quick fox", "fox jumped", "jumped over", "over the", "the lazy", "lazy dog"

1-skip-2-grams of "the quick fox jumped over the lazy dog"  
"the quick", "quick fox", "fox jumped", "jumped over", "over the", "the lazy", "lazy dog"
"the fox", "quick jumped", "fox over", "jumped the", "over lazy", "the dog"

The skip-gram approach tries to predict the context from the word.

The other common technique is CBOW (continuous bag of words) which keeps a set of the words around the subject word. The CBOW approach tries to predict a word from its context.

Both of these techniques rely on statistical NLP rule of thumb: "You shall know a word by the company it keeps".

Apache Spark's technique is based on the following paper  
https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

In [8]:
var word2Vec = new Word2Vec()
word2Vec = word2Vec.setInputCol("words")
word2Vec = word2Vec.setOutputCol("vectorfeatures")
word2Vec = word2Vec.setSeed(123)

// Parameters set with their defaults
word2Vec = word2Vec.setVectorSize(100)
word2Vec = word2Vec.setWindowSize(5)
word2Vec = word2Vec.setNumPartitions(1)
word2Vec = word2Vec.setMinCount(5)
word2Vec = word2Vec.setStepSize(0.025)
word2Vec = word2Vec.setMaxIter(1)

In [9]:
val model = word2Vec.setVectorSize(15).setMinCount(20).fit(dataframe)
val word2vecDF = model.transform(dataframe)

In [10]:
word2vecDF

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, isTopLevel: double, textLength: int, avgSentenceLengths: double, words: array<string>, numEntities: double, score: double, isStoryPred: string, popularity: string, vectorfeatures: vector]

In [12]:
model.getVectors.show(5)

+---------+--------------------+
|     word|              vector|
+---------+--------------------+
|  serious|[-0.0754927694797...|
|forgotten|[-0.0042912219651...|
|     rate|[0.04590357840061...|
|     rage|[-0.0390916541218...|
|     snow|[0.00373126356862...|
+---------+--------------------+
only showing top 5 rows



In [13]:
val word2VecLookup = model.getVectors.collect().map(r => (r.getString(0) -> r.getAs[Vector](1))).toMap

It looks like the most similar words are not terribly close, but let's see if can still find a way to expand our word list.

In [14]:
// this produces 20 words ranked by cosine similarity, which measure similarity between two vectors
// for more info on cosine similarity https://en.wikipedia.org/wiki/Cosine_similarity
model.findSynonyms("magic", 20).show(20)

+-------+-------------------+
|   word|         similarity|
+-------+-------------------+
|  angel| 0.3053628863267937|
|univers|0.28363234065074727|
|   solv| 0.2809488776242423|
|  idiot|0.27815078267075194|
|  twice| 0.2749431782796586|
|  scare| 0.2746737418391509|
| effort| 0.2745457401840115|
|   seri|0.27072942789235316|
|     dr| 0.2705888966650402|
| strong| 0.2681746477120919|
|collect|0.26666633944080287|
| prayer|0.26638082807525315|
|   forc|0.26501476252105094|
|control| 0.2628084372101809|
|  model| 0.2602417259917204|
| mistak| 0.2601525169906674|
| darwin|0.25777094083049235|
|  you'v| 0.2576953688481371|
|      5|0.25737305634634916|
|   lord| 0.2572881943404546|
+-------+-------------------+



Now let's load our word list, we will have to stem the tokens in order to make sure they match the vocabulary of the model.

In [15]:
import opennlp.tools.stemmer.PorterStemmer
val stemmer = new PorterStemmer()

$$avg(cossim(w, V)) = \frac{\sum_{v \in V}{\frac{w \bullet v}{||w||\ ||v||}}}{|V|}$$
$$avg(cossim(w, V)) = \frac{w \bullet \sum_{v \in V}{\frac{v}{||v||}}}{||w||\ |V|}$$
$$avg(cossim(w, V)) = \frac{w}{||w||} \bullet \frac{\sum_{v \in V}{\frac{v}{||v||}}}{|V|}$$

In [16]:
// let's define some useful methods

def dot(u: Vector, v: Vector): Double = {
  require(u.size == v.size)
  u.toArray.zip(v.toArray).map { case (ui, vi) => ui * vi }.sum
}

def addVec(u: Vector, v: Vector): Vector = {
  require(u.size == v.size)
  Vectors.dense(u.toArray.zip(v.toArray).map { case (ui, vi) => ui + vi })
}

def scalarMultVec(v: Vector, s: Double): Vector = Vectors.dense(v.toArray.map(_ * s))

In [17]:
// first we load the words and their vectors
val scifiwords = scala.io.Source.fromFile("scifiwordlist.txt").getLines.flatMap(_.split(" |-")).toSet.flatMap {
  word: String =>
    val stem = stemmer.stem(word)
    word2VecLookup.get(stem).map(v => (stem -> v))
}.toMap

In [18]:
// now we make the part of the formula that is just dependent on the vocabulary
val scifiAvg = scifiwords.values.tail.foldLeft(
  scalarMultVec(scifiwords.values.head, 1/(scifiwords.size * Vectors.norm(scifiwords.values.head, 2.0)))
) {
  case (accVec, vec) =>
    addVec(accVec, scalarMultVec(vec, 1/(scifiwords.size * Vectors.norm(vec, 2.0))))
}

In [19]:
// now we produce the average cosine similarity between our words that are not already in the scifi vocabulary 
// and the scifi vocabulary
word2VecLookup.collect {
  case (word, vector) if !scifiwords.contains(word) =>
    (word, dot(scalarMultVec(vector, Vectors.norm(vector, 2.0)), scifiAvg))
}.toSeq.sortBy(-_._2).take(20).foreach(println)

(krab,0.7012258538610338)
(squidward,0.6385781826256198)
(spongebob,0.5223763730907264)
(michael,0.4094800493445954)
(kevin,0.3568490862287379)
(god,0.3354935887543298)
(fought,0.3199144713136879)
(citi,0.24619461641136153)
(tap,0.2402533442041527)
(fate,0.20674418142077564)
(war,0.20563222546366755)
(don,0.19895481704753792)
(presid,0.19793062565537214)
(race,0.19449078988730695)
(human,0.19141599651801783)
(struggl,0.1873970353267566)
(knock,0.18279471613435766)
(appreci,0.17717906184209886)
(bori,0.17705121730246276)
(prayer,0.1767445088301382)


Now we do the same for the fantasy vocabulary

In [20]:
val fantasywords = scala.io.Source.fromFile("fantasywordlist.txt").getLines.flatMap(_.split(" |-")).toSet.flatMap {
  word: String =>
    val stem = stemmer.stem(word)
    word2VecLookup.get(stem).map(v => (stem -> v))
}.toMap

In [21]:
// now we make the part of the formula that is just dependent on the vocabulary
val fantasyAvg = fantasywords.values.tail.foldLeft(
  scalarMultVec(fantasywords.values.head, 1/(fantasywords.size * Vectors.norm(fantasywords.values.head, 2.0)))
) {
  case (accVec, vec) =>
    addVec(accVec, scalarMultVec(vec, 1/(scifiwords.size * Vectors.norm(vec, 2.0))))
}

In [22]:
word2VecLookup.collect {
  case (word, vector) if !fantasywords.contains(word) =>
    (word, dot(scalarMultVec(vector, Vectors.norm(vector, 2.0)), fantasyAvg))
}.toSeq.sortBy(-_._2).take(20).foreach(println)

(krab,0.2826377543052795)
(squidward,0.25965435594790937)
(spongebob,0.21350200068648345)
(michael,0.16858250919368056)
(kevin,0.1508679072363573)
(god,0.14955804994001423)
(fought,0.1348145943757862)
(citi,0.10669752214612108)
(tap,0.09939386460345767)
(fate,0.0900982715448498)
(dai,0.08929908031546945)
(war,0.08924853892577325)
(planet,0.08502878985482892)
(don,0.08490447251678901)
(human,0.08307159389491206)
(race,0.0824396718582599)
(struggl,0.08182575730191584)
(presid,0.08160450453857307)
(life,0.0798570300304728)
(knock,0.07792270693331885)


It seems that perhaps a human-in-the-loop method would work best here. There are definitely some words that would be appropriate additions. Feel free to add algorithmically or manually to the wordlists.

### Exercise 4. predict the score using the vector features

In [23]:
val Array(trainScoreDF, testScoreDF) = word2vecDF.randomSplit(Array(0.75, 0.25), 123)

In [24]:
var labelIndexerScore = new StringIndexer()
labelIndexerScore = labelIndexerScore.setInputCol("popularity")
labelIndexerScore = labelIndexerScore.setOutputCol("popularityLabel")
val labelIndexScoreModel = labelIndexerScore.fit(trainScoreDF)
labelIndexScoreModel.labels.toList

List(not-popular, popular)

In [25]:
var hashingTF = new HashingTF()
hashingTF = hashingTF.setInputCol("words")
hashingTF = hashingTF.setOutputCol("wordfeatures")
hashingTF = hashingTF.setNumFeatures(25)

var idf = new IDF()
idf = idf.setInputCol("wordfeatures")
idf = idf.setOutputCol("tfidffeatures")

var vectorAssemblerScore = new VectorAssembler()
vectorAssemblerScore = vectorAssemblerScore.setInputCols(Array(
  "textLength",
  "avgSentenceLengths", 
  "numEntities",
  "tfidffeatures",
  "vectorfeatures"
))
vectorAssemblerScore = vectorAssemblerScore.setOutputCol("features")

var classifierScore = new DecisionTreeClassifier()
classifierScore = classifierScore.setFeaturesCol("features")
classifierScore = classifierScore.setLabelCol("popularityLabel")

var labelConverterScore = new IndexToString()
labelConverterScore = labelConverterScore.setInputCol("prediction")
labelConverterScore = labelConverterScore.setOutputCol("popularityPred")
labelConverterScore = labelConverterScore.setLabels(labelIndexScoreModel.labels)

var pipelineScore = new Pipeline().setStages(Array(
  labelIndexerScore,
  hashingTF,
  idf,
  vectorAssemblerScore,
  classifierScore,
  labelConverterScore
))

In [26]:
val modelScore = pipelineScore.fit(trainScoreDF)

In [27]:
val predictionsScore = modelScore.transform(testScoreDF)

In [28]:
predictionsScore

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, isTopLevel: double, textLength: int, avgSentenceLengths: double, words: array<string>, numEntities: double, score: double, isStoryPred: string, popularity: string, vectorfeatures: vector, popularityLabel: double, wordfeatures: vector, tfidffeatures: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double, popularityPred: string]

In [29]:
var evaluatorScore = new BinaryClassificationEvaluator()
evaluatorScore = evaluatorScore.setLabelCol("popularityLabel")
evaluatorScore = evaluatorScore.setRawPredictionCol("rawPrediction")
val auc = evaluatorScore.evaluate(predictionsScore)
println("AUC = " + auc)

AUC = 0.5034774436090226


In [30]:
predictionsScore.select("popularityPred", "popularity").rdd.map {
  row =>
    s"truth: ${row.getString(1)}, prediction: ${row.getString(0)}"
}.groupBy(identity).mapValues(_.size).collect().foreach(println)

(truth: not-popular, prediction: not-popular,72)
(truth: not-popular, prediction: popular,103)
(truth: popular, prediction: not-popular,54)
(truth: popular, prediction: popular,98)


### end of exercise